In [ ]:
#!pip install robosuite
#!pip install mujoco
#!pip install h5py
#pip install gymansium==1.2.0
...

In [ ]:
import robosuite as suite
from robosuite.wrappers import GymWrapper
import gymnasium as gym
from gymnasium.vector import AsyncVectorEnv
from gymnasium.wrappers import Autoreset
import torch
from torch import Tensor
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
from IPython.display import clear_output
from dataclasses import dataclass
from copy import deepcopy
clear_output()

In [ ]:
@dataclass
class Hypers:
    num_env = 1
    horizon = 1000
    lr = 3e-4
    lambda_ = 0
    tau = 0
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

hypers = Hypers()

env_configs = {
    "robots":["Panda"],
    "gripper_types":["JacoThreeFingerDexterousGripper"],
    "has_renderer":True,
    "use_camera_obs":False,
    "has_offscreen_renderer":False,
    "horizon":hypers.horizon, 
}

def vec_env():
    def make_env():
        x = suite.make(env_name ="PickPlace" ,**env_configs)
        x = GymWrapper(x)
        x = Autoreset(x)
        return x
    return AsyncVectorEnv([make_env for _ in range(hypers.num_env)])

 
clear_output()

In [ ]:
e = vec_env()

In [ ]:
class Actor(nn.Module):
    def __ini__(self):
        super().__init__(self)

    def forward(x:Tensor):
        pass

class Critic(nn.Module):
    def __ini__(self):
        super().__init__(self)

    def forward(x:Tensor):
        pass

In [ ]:
env.reset()
for i in range(1000000):
    action = env.action_space.sample()
    obs, reward, done, info,_ = env.step(action)  
    env.render()  
env.close()
...